In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

/Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/22/25 06:42:18] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=323511;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=637889;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ansul/Library/Application Support/sagemaker/config.yaml


In [9]:
sm_boto3 = boto3.client("sagemaker", region_name="us-east-1")  # or your preferred region
sess = sagemaker.Session(boto3.Session(region_name="us-east-1"))
region = sess.boto_region_name
bucket = 'awssagemaker22'
print("Using bucket " + bucket) 

[02/22/25 07:05:06] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=468850;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=818387;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/botocore/credentials.py#1278\1278]8;;\

Using bucket awssagemaker22


In [10]:
df = pd.read_csv('mob_price_classification_train.csv')

In [11]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [13]:
df.shape

(2000, 21)

In [14]:
df['price_range'].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [15]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [16]:
df.shape

(2000, 21)

In [17]:
df.isnull().mean() * 100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [18]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [19]:
label = features.pop(-1)
label

'price_range'

In [20]:
x = df[features]
y = df[label]

In [21]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [22]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [23]:
x.shape

(2000, 20)

In [24]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [25]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=0)

In [26]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1700, 20) (300, 20) (1700,) (300,)


In [27]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test




In [28]:
print(trainX.shape, testX.shape)

(1700, 21) (300, 21)


In [29]:
trainX.head()
testX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
405,1454,1,0.5,1,1,0,34,0.7,83,4,...,250,1033,3419,7,5,5,1,1,0,3
1190,1092,1,0.5,1,10,0,11,0.5,167,3,...,468,571,737,14,4,11,0,1,0,0
1132,1524,1,1.8,1,0,0,10,0.6,174,4,...,154,550,2678,16,5,13,1,0,1,2
731,1807,1,2.1,0,2,0,49,0.8,125,1,...,337,1384,1906,17,13,13,0,1,1,2
1754,1086,1,1.7,1,0,1,43,0.2,111,6,...,56,1150,3285,11,5,17,1,1,0,2


In [30]:
trainX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [31]:
testX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [32]:
trainX.to_csv('train_Version_1.csv', index=False)
testX.to_csv('testX_Version_1.csv', index=False)

In [33]:
sk_prefix = 'sagemaker/mobile_price_classification/sklearncontainer'
trainpath = sess.upload_data(
    path='train_Version_1.csv', bucket=bucket, key_prefix=sk_prefix
)
testpath = sess.upload_data(
    path='testX_Version_1.csv', bucket=bucket, key_prefix=sk_prefix
)
print(trainpath, testpath)

s3://awssagemaker22/sagemaker/mobile_price_classification/sklearncontainer/train_Version_1.csv s3://awssagemaker22/sagemaker/mobile_price_classification/sklearncontainer/testX_Version_1.csv


In [34]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train_Version_1.csv")
    parser.add_argument("--test-file", type=str, default="testX_Version_1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose = 3,n_jobs=-1)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Writing script.py


In [45]:
import boto3
import sagemaker
from sagemaker.sklearn.estimator import SKLearn

# Set up the session with explicit region
boto_session = boto3.Session(region_name='eu-north-1')  # or your preferred region
sagemaker_session = sagemaker.Session(boto_session=boto_session)

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::120569620760:role/SageMakerExecutionRole",  # Use your SageMaker execution role ARN
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances=True,
    max_wait=7200,
    max_run=3600,
    sagemaker_session=sagemaker_session
)

[02/22/25 07:48:57] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=328384;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=130575;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/botocore/credentials.py#1278\1278]8;;\

In [46]:
sklearn_estimator.fit({'train': trainpath, 'test': testpath}, wait=True)

[02/22/25 07:48:59] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=976157;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=552948;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[02/22/25 07:49:02] INFO     Created S3 bucket: sagemaker-eu-north-1-120569620760                    ]8;id=798131;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=498243;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/session.py#723\723]8;;\

                    INFO     Creating training-job with name:                                       ]8;id=432867;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=719997;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/session.py#1042\1042]8;;\
                             RF-custom-sklearn-2025-02-22-02-18-59-168                                             

2025-02-22 02:19:06 Starting - Starting the training job...
2025-02-22 02:19:39 Downloading - Downloading input data...
2025-02-22 02:20:04 Downloading - Downloading the training image...
2025-02-22 02:20:45 Training - Training image download completed. Training in progress..2025-02-22 02:20:49,711 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-02-22 02:20:49,714 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-02-22 02:20:49,752 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-02-22 02:20:49,897 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-02-22 02:20:49,908 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-02-22 02:20:49,919 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-02-22 02:20:49,928 sagemaker-training-toolkit INFO     Invoking user script
Trai

In [48]:
sklearn_estimator.latest_training_job.wait(logs="None")

# Then get the model artifacts using the sagemaker session's boto client
artifact = sagemaker_session.boto_session.client('sagemaker').describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)['ModelArtifacts']['S3ModelArtifacts']

print("Model Artifacts: ", artifact)


2025-02-22 02:21:08 Starting - Preparing the instances for training
2025-02-22 02:21:08 Downloading - Downloading the training image
2025-02-22 02:21:08 Training - Training image download completed. Training in progress.
2025-02-22 02:21:08 Uploading - Uploading generated training model
2025-02-22 02:21:08 Completed - Training job completed
Model Artifacts:  s3://sagemaker-eu-north-1-120569620760/RF-custom-sklearn-2025-02-22-02-18-59-168/output/model.tar.gz


In [50]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::120569620760:role/SageMakerExecutionRole",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,  # Use the same version as in your estimator
    py_version="py3"  # Specify Python version
)

In [51]:
model

In [53]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::120569620760:role/SageMakerExecutionRole",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
    py_version="py3",
    sagemaker_session=sagemaker_session  # Add this line to use the existing session
)

endpoint_name = "Custom-sklearn-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint Name:{} ".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name
)

Endpoint Name:Custom-sklearn-endpoint-2025-02-22-03-00-26 


[02/22/25 08:30:27] INFO     Creating model with name: Custom-sklearn-model-2025-02-22-03-00-26     ]8;id=61605;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=638243;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/session.py#4094\4094]8;;\

[02/22/25 08:30:28] INFO     Creating endpoint-config with name                                     ]8;id=343674;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=652699;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/session.py#5889\5889]8;;\
                             Custom-sklearn-endpoint-2025-02-22-03-00-26                                           

[02/22/25 08:30:29] INFO     Creating endpoint with name                                            ]8;id=103664;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=47816;file:///Users/ansul/Desktop/AWSSagemaker/myenv/lib/python3.9/site-packages/sagemaker/session.py#4711\4711]8;;\
                             Custom-sklearn-endpoint-2025-02-22-03-00-26                                           

-----!

In [54]:
predictor

In [55]:
endpoint_name

'Custom-sklearn-endpoint-2025-02-22-03-00-26'

In [57]:
testX[features][0:2].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0]]

In [58]:
print(predictor.predict(testX[features][0:2].values.tolist()))

[3 0]


In [62]:
# List all endpoints
endpoints = sm_boto3.list_endpoints()
print("Available endpoints:")
for endpoint in endpoints['Endpoints']:
    print(f"- {endpoint['EndpointName']}")

# Then delete the specific endpoint if it exists
try:
    sm_boto3.delete_endpoint(EndpointName=endpoint_name)
    print(f"Successfully deleted endpoint: {endpoint_name}")
except sm_boto3.exceptions.ClientError as e:
    if "Could not find endpoint" in str(e):
        print(f"Endpoint {endpoint_name} does not exist")
    else:
        raise e

Available endpoints:
Endpoint Custom-sklearn-endpoint-2025-02-22-03-00-26 does not exist
